# Mix-in

* Mix-in
    * Rubyで多重継承を実現する機能の事。Rubyの代表的な機能
* Rubyの継承は単一継承のみ許可
    * <font color="red">同時に複数のスーパークラスを持つ「多重継承」</font>は許されていない
    * class式で指定できるスーパークラスは必ず一つだけ
* あるクラスが持つ機能を他のクラスに適用する場合
    * Rubyは、<font color="red">クラスに機能を混ぜ合わせる事で複数のクラスで機能を共有する機能(Mix-in)</font>で実現する
    * <font color="red">複数のクラスで適用したい機能は、モジュールというオブジェクトで定義する</font>
        * これをクラスに取り込むことで機能を拡張する
* モジュールとクラスの相違点
    * モジュールは単独ではインスタンス化できない(newできない)
    * モジュールは継承できない
    * モジュールは他のクラスやモジュールに取り込むことができる
* モジュールの目的
    * 名前空間の提供
    * Mix-inにおける機能の定義

In [1]:
### モジュールの定義。クラスとの違いはclassの代わりにmoduleを使っているだけ。
module Bar
  def methodA
    @a
  end
end

puts Bar.ancestors
puts Bar.instance_methods

### newはできない
puts Bar.new

[Bar]
[:methodA]


NoMethodError: undefined method `new' for Bar:Module

## モジュールのクラスへの取り込み

* includeメソッドを使う
* モジュールとancestorsの表示について
    * ` [FooExt, Bar, Foo, ... `
        * 表示は上のようになっているが、BarモジュールのスーパークラスにFooがあるわけでは無い
            * そもそもモジュールにスーパークラスは無い
        * FooExt : クラス
        * Bar : FooExtがincludeしているモジュール
        * Foo : FooExtのスーパークラス
    * includeが実行されるとインタプリタは指定したモジュールに対応する無名クラスを作成して、スーパークラスとの間に組み込む
    * 無名クラス
        * モジュールをincludeしたクラスはモジュールのメソッドを使える
        * モジュール名のクラス(Barクラス)ができているわけでは無く、無名のクラスのメソッドとして使っている
    * 無名クラスはRubyインタプリタの実装の都合上作られる
        * <font color="red">配列の順が継承チェーンの順を表しているわけではない</font>
    * ユーザーに意識させたくないので、superclassで参照できないようにしている

In [24]:
module Bar
  def methodA
    @a
  end
end

class Foo; end

### Fooクラスを継承したFooExtクラスにBarモジュールを取り込む
class FooExt < Foo
  include Bar
end

puts "### FooExtのスーパークラスはFoo(Barモジュールではない)"
puts FooExt.superclass
puts "### FooExtの親クラス(includeしているモジュール含む)を配列で表示"
puts FooExt.ancestors
puts ""

puts "### BarモジュールにmethodAがある"
puts Bar.instance_methods(false)
puts "### FooExtクラスはincludeしているBarモジュールのmethodAが利用可能"
puts FooExt.instance_methods

### FooExtのスーパークラスはFoo(Barモジュールではない)
Foo
### FooExtの親クラス(includeしているモジュール含む)を配列で表示
[FooExt, Bar, Foo, Object, PP::ObjectMixin, JSON::Ext::Generator::GeneratorMethods::Object, Kernel, BasicObject]

### BarモジュールにmethodAがある
[:methodA]
### FooExtクラスはincludeしているBarモジュールのmethodAが利用可能
[:methodB, :methodA, :pry, :__binding__, :pretty_print, :pretty_print_cycle, :pretty_print_instance_variables, :pretty_print_inspect, :to_json, :instance_of?, :public_send, :instance_variable_get, :instance_variable_set, :instance_variable_defined?, :remove_instance_variable, :private_methods, :kind_of?, :instance_variables, :tap, :public_method, :singleton_method, :is_a?, :extend, :define_singleton_method, :method, :to_enum, :enum_for, :<=>, :===, :=~, :!~, :eql?, :respond_to?, :freeze, :inspect, :display, :object_id, :send, :gem, :to_s, :pretty_inspect, :nil?, :hash, :class, :singleton_class, :clone, :dup, :itself, :taint, :tainted?, :untaint, :untrust, :trust, :untrusted?, :methods, :protected_methods, :froze

## 複数モジュールをincludeした場合のメソッドの優先順位

* 後にincludeした方が優先される

In [30]:
module A
  def methodC
    print "moduleA methodC"
  end
end

module B
  def methodC
    print "moduleB methodC"
  end
end

class Hoge
  include A
  include B
end

hoge = Hoge.new
puts "後でincludeしたモジュールBのmethodCが実行される"
puts hoge.methodC
puts Hoge.ancestors

後でincludeしたモジュールBのmethodCが実行される
moduleB methodC
[Hoge, B, A, Object, PP::ObjectMixin, JSON::Ext::Generator::GeneratorMethods::Object, Kernel, BasicObject]
